# Video of brain spinning with all clusters, organized by insertion

In [5]:
from brainwidemap import bwm_query, bwm_units
from one.api import ONE

one = ONE(base_url='https://openalyx.internationalbrainlab.org')
# Dataframe with info on all sessions and probes released for the BWM
bwm_df = bwm_query(one)
# Dataframe with information on all neurons used in the analyses in the BWM paper
unit_df = bwm_units(one)

c:\Users\Dan\anaconda3\envs\iblenv\lib\site-packages\neurodsp\__init__.py:5: FutureWarning: neurodsp has been renamed to ibldsp and the old name will be deprecated on 01-Sep-2024.
  warn(
c:\Users\Dan\anaconda3\envs\iblenv\lib\site-packages\ibllib\atlas\__init__.py:202: DeprecationWarning: ibllib.atlas is deprecated. Please install iblatlas using "pip install iblatlas" and use this module instead
  warnings.warn('ibllib.atlas is deprecated. Please install iblatlas using "pip install iblatlas" and use '


Loading bwm_query results from fixtures/2023_12_bwm_release.csv
Loading bwm_query results from fixtures/2023_12_bwm_release.csv


c:\Users\Dan\anaconda3\envs\iblenv\lib\site-packages\ibllib\atlas\atlas.py:13: DeprecationWarning: ibllib.atlas.regions.BrainRegions is deprecated. Use iblatlas.regions.BrainRegions instead
  warnings.warn(warning_text, DeprecationWarning)


In [ ]:

from pathlib import Path

from one.api import ONE
import ephys_atlas.data

LABEL = '2024_W04'  # 2023_W51_autism for example would return the autism dataset features
LOCAL_DATA_PATH = Path('C:/scratch/')
one = ONE(base_url="https://alyx.internationalbrainlab.org", mode='local')

df_raw_features, df_clusters, df_channels, df_probes = ephys_atlas.data.download_tables(
    label=LABEL, local_path=LOCAL_DATA_PATH, one=one)
# df_raw_features, df_clusters, df_channels = load_tables(local_path=FOLDER_GDRIVE)

# df_depths = ephys_atlas.data.compute_depth_dataframe(df_raw_features, df_clusters, df_channels)
df_voltage = df_raw_features.merge(df_channels, left_index=True, right_index=True)


## %%
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure()
sns.countplot(data=df_clusters, x='label',  palette='deep')

In [6]:
bwm_df.loc[bwm_df['lab'] == 'hoferlab', 'lab'] = 'hoferflogellab'
bwm_df.loc[bwm_df['lab'] == 'mrsicflogellab', 'lab'] = 'hoferflogellab'

In [7]:
from iblatlas import atlas
aa = atlas.AllenAtlas()
from iblatlas.regions import BrainRegions
br = BrainRegions()
import numpy as np
import pandas as pd
import time
import seaborn as sns

In [8]:
def hex_to_rgb(hex_color):
    # Remove '#' if present
    if hex_color.startswith('#'):
        hex_color = hex_color[1:]

    # Convert hex to RGB
    red = int(hex_color[0:2], 16) / 255.0
    green = int(hex_color[2:4], 16) / 255.0
    blue = int(hex_color[4:6], 16) / 255.0

    return red, green, blue

In [9]:
ulabs = np.unique(bwm_df.lab)

In [10]:
lab_data = []

id2color = {}

for lab in ulabs:
    pids = bwm_df.pid[bwm_df.lab == lab]
    
    # print((lab, len(pids)))


    coords = []

    for pid in pids:
        units = unit_df[unit_df.pid==pid]
        # print(len(units))
        for i, unit in units.iterrows():
            coord = aa.xyz2ccf([unit.x, unit.y, unit.z])
            if unit.atlas_id in id2color.keys():
                color = id2color[unit.atlas_id]
            else:
                color = hex_to_rgb(aa.regions.hexcolor[aa.regions.id2index(unit.atlas_id)[1][0][0]])
                id2color[unit.atlas_id] = color
            coords.append([coord, color])

    lab_df = pd.DataFrame(columns=['mlapdv', 'color'], data=coords)

    lab_data.append(lab_df)

In [11]:
# contributions
print(ulabs)

['angelakilab' 'churchlandlab' 'churchlandlab_ucla' 'cortexlab' 'danlab'
 'hausserlab' 'hoferflogellab' 'mainenlab' 'steinmetzlab' 'wittenlab'
 'zadorlab']


In [12]:
# institution_map = {'cortexlab': 'UCL', 'mainenlab': 'CCU', 'zadorlab': 'CSHL (Z)',
#                        'churchlandlab': 'CSHL (C)', 'angelakilab': 'NYU',
#                        'wittenlab': 'Princeton', 'hoferlab': 'SWC', 'mrsicflogellab': 'SWC',
#                        'danlab': 'Berkeley', 'steinmetzlab': 'UW', 'churchlandlab_ucla': 'UCLA', 'hausserlab':'UCL'}
institution_map = {'cortexlab': 'Carandini & Harris Labs (University College London)', 'mainenlab': 'Mainen Lab (Champalimaud Center for the Unknown)',
                   'zadorlab': 'Zador Lab (Cold Spring Harbor)', 'churchlandlab': 'Churchland Lab (Cold Spring Harbor)',
                   'angelakilab': 'Angelaki Lab (New York University)', 'wittenlab': 'Witten Lab (Princeton University)', 
                   'hoferflogellab': 'Hofer & Mrsic-Flogel Labs (Sainsbury-Wellcome Centre)',
                   'danlab': 'Dan Lab (University of California, Berkeley)', 'steinmetzlab': 'Steinmetz Lab (University of Washington)',
                   'churchlandlab_ucla': 'Churchland Lab (University of California, Los Angeles)', 'hausserlab': 'Hausser Lab (University College London)'}

In [13]:
names_clean = [institution_map[lab] for lab in ulabs]

In [14]:
names_clean

['Angelaki Lab (New York University)',
 'Churchland Lab (Cold Spring Harbor)',
 'Churchland Lab (University of California, Los Angeles)',
 'Carandini & Harris Labs (University College London)',
 'Dan Lab (University of California, Berkeley)',
 'Hausser Lab (University College London)',
 'Hofer & Mrsic-Flogel Labs (Sainsbury-Wellcome Centre)',
 'Mainen Lab (Champalimaud Center for the Unknown)',
 'Steinmetz Lab (University of Washington)',
 'Witten Lab (Princeton University)',
 'Zador Lab (Cold Spring Harbor)']

In [15]:
names_contribs = ['Julius Benson, Jean-Paul Noel',
                  'Anne Urai',
                  'Anup Khanal, Felicia Davatolhagh',
                  'Karolina Socha',
                  'Fei Hu',
                  'Petrina Lau',
                  'Nathaniel Miska',
                  'Guido Meijer',
                  'Noam Roth',
                  'Alejandro Pan-Vazquez',
                  'Christpher Krasniak']

In [16]:
lab_colors = np.concatenate([sns.color_palette("Dark2"), sns.color_palette('Set2')])

In [17]:
import oursin as urchin
urchin.setup(id='test')

(URN) connected to server
Login sent with ID: test, copy this ID into the renderer to connect.


In [108]:
# Load root
urchin.ccf25.load()

(Warning) Atlas was already loaded, the renderer can have issues if you try to load an atlas twice.


In [109]:
urchin.ccf25.root.set_visibility(True, side=urchin.utils.Side.FULL)
urchin.ccf25.root.set_color([0,0,0])
urchin.ccf25.root.set_material('transparent-lit')
urchin.ccf25.root.set_alpha(0.15)

In [110]:
urchin.camera.main.set_rotation([-45, 0, 0])
urchin.camera.set_brain_rotation(225)
urchin.camera.main.set_zoom(50)
urchin.camera.main.set_mode('perspective')

In [111]:
text = urchin.text.Text()
text.set_position([-0.5, -0.75])
text.set_color('#ffffff')
text.set_font_size(64)
text.set_position([-0.92, -0.75])

contrib_text = urchin.text.Text()
contrib_text.set_position([-0.5, -0.85])
contrib_text.set_color('#ffffff')
contrib_text.set_font_size(48)
contrib_text.set_position([-0.92, -0.85])

# text.set_text("test")
# contrib_text.set_text("test")

In [112]:
# Pre-create all neurons and store them in lists
all_meshes = []
all_colors = []
neurons_by_lab = {}

for i, lab_df in enumerate(lab_data):
    
    ccolor = list(lab_colors[i])
    meshes = urchin.meshes.create(len(lab_df))
    all_meshes = all_meshes + meshes
    positions = []
    colors = []
    
    for j, row in lab_df.iterrows():
        positions.append([row.mlapdv[1], row.mlapdv[0], row.mlapdv[2]])
        colors.append(ccolor)
        all_colors.append(row.color)
    
    urchin.meshes.set_positions(meshes, positions)
    urchin.meshes.set_colors(meshes, colors)
    urchin.meshes.set_scales(meshes, [[0,0,0]]*len(meshes))

    neurons_by_lab[i] = meshes

In [113]:
s_per_build = 2
s_per_lab = 3
s_pause = 1
s_end = 6
fps = 30

lab_frames = fps*len(ulabs)*s_per_lab
pause_frames = lab_frames + s_pause * fps
total_frames = pause_frames + s_end * fps
total_s = s_per_lab*len(ulabs) + s_pause + s_end
print((lab_frames, pause_frames, total_frames))

frames_per_build = s_per_build * fps
frames_per_lab = s_per_lab*fps
lab_meshes = []

pause_set = False
end_set = False

def set_view(frame):   
    global lab_idx, ccolor, lab_meshes, lab_frames, pause_frames, pause_set, pause_frames, end_set

    urchin.camera.set_brain_rotation(225 + frame * 0.75)
    
    if (frame < lab_frames):
        if np.mod(frame, frames_per_lab) == 0:
            if lab_meshes:
                urchin.meshes.set_scales(lab_meshes, [[0,0,0]]*len(lab_meshes))

            lab_idx = int(np.floor(frame / frames_per_lab))
            ccolor = list(lab_colors[lab_idx])

            text.set_text(names_clean[lab_idx])
            text.set_color(ccolor)

            contrib_text.set_text(names_contribs[lab_idx])
            contrib_text.set_color(ccolor)

            lab_meshes = neurons_by_lab[lab_idx]

        perc = np.min([np.mod(frame, frames_per_lab) / frames_per_build, 1])
        num_meshes = int(np.floor(len(lab_meshes) * perc))

        urchin.meshes.set_scales(lab_meshes[0:num_meshes], [[0.04, 0.04, 0.04]]*num_meshes)
    
    elif (frame < pause_frames):
        if not pause_set:
            pause_set = True

            text.set_text("")
            contrib_text.set_text("")
            
            urchin.meshes.set_scales(lab_meshes, [[0, 0, 0]]*len(lab_meshes))

    else:
        if not end_set:
            end_set = True

            text.set_text("International Brain Laboratory")
            text.set_color([0,0,0])
            contrib_text.set_text("Brain Wide Map")
            contrib_text.set_color([0,0,0])
            urchin.meshes.set_colors(all_meshes, all_colors)
            urchin.meshes.set_scales(all_meshes, [[0.04, 0.04, 0.04]]*len(all_meshes))

all_meshes = []
for i in np.arange(0,len(neurons_by_lab)):
    all_meshes = all_meshes + neurons_by_lab[i]

(990, 1020, 1200)


In [79]:
# test mode
for i in np.arange(0, total_frames):
    if np.mod(i, 10)==0:
        set_view(i)
        time.sleep(0.2)

In [114]:
await urchin.camera.main.capture_video(file_name='mov.mp4', callback=set_view, duration=total_s)

(Camera receive) Camera CameraMain received an image
(Camera receive) CameraMain complete
(Camera receive) Camera CameraMain received an image
(Camera receive) CameraMain complete
(Camera receive) Camera CameraMain received an image
(Camera receive) CameraMain complete
(Camera receive) Camera CameraMain received an image
(Camera receive) CameraMain complete
(Camera receive) Camera CameraMain received an image
(Camera receive) CameraMain complete
(Camera receive) Camera CameraMain received an image
(Camera receive) CameraMain complete
(Camera receive) Camera CameraMain received an image
(Camera receive) CameraMain complete
(Camera receive) Camera CameraMain received an image
(Camera receive) CameraMain complete
(Camera receive) Camera CameraMain received an image
(Camera receive) CameraMain complete
(Camera receive) Camera CameraMain received an image
(Camera receive) CameraMain complete
(Camera receive) Camera CameraMain received an image
(Camera receive) CameraMain complete
(Camera re

CancelledError: 

(Camera receive) Camera CameraMain received an image
